In [1]:
# 딥러닝

# 피드포워드(feed forward) 동작방식
# 입력층으로 데이터가 입력되고, 1개 이상으로 구성되는 은닉층을 거쳐서 마지막에 출력층으로 출력값을 내보내는 과정
# 딥러닝에서는 이전 층에서 나온 출력값 => 층과 층 사이에 적용되는 가중치에 영향을 받은 다음, 다음 층의 입력값으로 들어가는 것을 의미함
# 딥러닝 입력층에서는 활성화 함수인 sigmoid 를 적용하지 않고, 입력값 그대로 출력으로 내보내는 것이 관례화 되어 있음
# 딥러닝에서는 출력층에서의 출력값 y와 정답 t(트레이닝 데이터)와의 차이를 이용하여 오차가 최소가 되도록 각 층에 있는 가중치와 바이어스를 최적화


In [15]:
# XOR 문제
# 1개의 로지스틱 회귀로는 XOR 구현 불가
# NAND OR AND 조합으로 XOR 문제 해결
# 딥러닝으로 위의 조합 없이 해결 가능?

# LogicGate class 딥러닝 버전
import numpy as np

def sigmoid(x):
    return 1 / (1+np.exp(-x))

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        
        # 입력데이터, 정답데이터 초기화
        self.__xdata = xdata.reshape(4,2) # 4개(2개 입력데이터 조합)의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1) # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬(정답은 1개)
        
        # 2층 hidden layer unit : 6개 가정, 가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,6) # weight, 2x6 matrix
        self.__b2 = np.random.rand(6)
        
        # 3층 output layer unit : 1개 가정, 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(6,1) # weight, 6x1 matrix, 1x2 2x6 6x1 => 1x1
        self.__b3 = np.random.rand(1)
        
        # 학습율 초기화
        self.__learning_rate = 1e-2 # 발산할 경우 더 작게 설정

    def feed_forward(self): # feed forward 를 통하여 손실함수값 계산(cross-entropy)
        
        delta = 1e-7
        
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2 = sigmoid(z2) # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀
        y = a3 = sigmoid(z3)
        
        # cross-entropy
        return -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1-y) + delta))
        
        
    def loss_val(self): # feed forward 를 통하여 손실함수값 계산(cross-entropy)
        
        delta = 1e-7
        
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2 = sigmoid(z2) # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀
        y = a3 = sigmoid(z3)
        
        # cross-entropy
        return -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1-y) + delta))
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될 때까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("init loss val = ", self.loss_val())
        
        for step in range(10001):
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
            
            if (step%500 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
    # 미래 값 예측
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2 = sigmoid(z2) # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀
        y = a3 = sigmoid(z3)
        
        if y > 0.5 :
            result = 1 # True
        else:
            result = 0 # False
        
        return y, result

In [16]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([0,0,0,1])

and_obj = LogicGate("AND", xdata, tdata)
and_obj.train()

init loss val =  7.173887350915879
step =  0 loss value =  6.867231519374462
step =  500 loss value =  2.1364270751165897
step =  1000 loss value =  1.942884221162047
step =  1500 loss value =  1.6351119933421747
step =  2000 loss value =  1.2275198072625482
step =  2500 loss value =  0.8247494267917819
step =  3000 loss value =  0.5479606974688658
step =  3500 loss value =  0.378660282176827
step =  4000 loss value =  0.2744676565444511
step =  4500 loss value =  0.20850186797858894
step =  5000 loss value =  0.1648393209433276
step =  5500 loss value =  0.13454825286818559
step =  6000 loss value =  0.11263493601197559
step =  6500 loss value =  0.09621446135572069
step =  7000 loss value =  0.08354547103458837
step =  7500 loss value =  0.0735303735053488
step =  8000 loss value =  0.06544984309682567
step =  8500 loss value =  0.058816104912713635
step =  9000 loss value =  0.053288457573629766
step =  9500 loss value =  0.04862271840898155
step =  10000 loss value =  0.04463989497

In [22]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([0,1,1,1])

or_obj = LogicGate("OR", xdata, tdata)
or_obj.train()

init loss val =  2.723303401054899
step =  0 loss value =  2.7060528095864456
step =  500 loss value =  1.9231042876083881
step =  1000 loss value =  1.5788071712282732
step =  1500 loss value =  1.1095069770292967
step =  2000 loss value =  0.6965801362957464
step =  2500 loss value =  0.43784615570032537
step =  3000 loss value =  0.2922830328896318
step =  3500 loss value =  0.20852417818939667
step =  4000 loss value =  0.15728300634074252
step =  4500 loss value =  0.12390437091231825
step =  5000 loss value =  0.10094848928276398
step =  5500 loss value =  0.08443680989431572
step =  6000 loss value =  0.07211616090929122
step =  6500 loss value =  0.06264084364862685
step =  7000 loss value =  0.05516867338612105
step =  7500 loss value =  0.04915081975210655
step =  8000 loss value =  0.044216917694381415
step =  8500 loss value =  0.040109371109705534
step =  9000 loss value =  0.036644248662165875
step =  9500 loss value =  0.033687165155721295
step =  10000 loss value =  0.0

In [18]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
tdata = np.array([0,1,1,0])

xor_obj = LogicGate("XOR", xdata, tdata)
xor_obj.train()

init loss val =  5.207526016577523
step =  0 loss value =  5.097841241310404
step =  500 loss value =  2.781187691220583
step =  1000 loss value =  2.7757037565613643
step =  1500 loss value =  2.771268598662169
step =  2000 loss value =  2.7669123973668692
step =  2500 loss value =  2.761755355444903
step =  3000 loss value =  2.7546679999445365
step =  3500 loss value =  2.7439062380591066
step =  4000 loss value =  2.726727150584591
step =  4500 loss value =  2.699178331698132
step =  5000 loss value =  2.656212987001057
step =  5500 loss value =  2.5920088993366948
step =  6000 loss value =  2.5014093998894404
step =  6500 loss value =  2.384120547326407
step =  7000 loss value =  2.247265758777826
step =  7500 loss value =  2.0997444327402253
step =  8000 loss value =  1.9446916412881612
step =  8500 loss value =  1.7785931821268461
step =  9000 loss value =  1.596535251114726
step =  9500 loss value =  1.3996210795782282
step =  10000 loss value =  1.1982669048017769


In [19]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.1449849]), 0)
(array([0.74254661]), 1)
(array([0.75055443]), 1)
(array([0.36683141]), 0)


In [21]:
# 입력층, 1개 이상의 은닉층, 출력층을 가지는 딥러닝 설계 => 더 복접한 손글씨 인식 및 이미지 인식 해결 가능!